<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 914.1 kB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [ ]:
import pickle

with open(f"{base_path}conversations_with_one_shot_reasoning.pkl", 'rb') as file:
    conversations = pickle.load(file)

print("Len loaded data:", len(conversations))

Len loaded data: 18184


In [15]:
import pandas as pd

data = []

for id in conversations:
    comments = conversations[id]['comments']
    for comment in comments:
        if not(comment['body'] == '[deleted]' or comment['body'] == '[removed]'):
            has_relatedness = False
            for pair in conversations[id]['pairs']:
                if pair['reply_id'] == comment['id']:
                    data.append([comment['id'], id, pair['comment_id'], comment['readability_score'], comment['evidence'], comment['has_reasoning'], comment['has_reasoningone-shot'], comment['politness_score'], pair['relatedness_score'],
                                 comment['subjectivity'], comment['polarity'], comment['body']])
                    has_relatedness = True
                    break
            if not has_relatedness:
              data.append([comment['id'], id, None, comment['readability_score'], comment['evidence'], comment['has_reasoning'], comment['has_reasoningone-shot'], comment['politness_score'], None,
                           comment['subjectivity'], comment['polarity'], comment['body']])

df = pd.DataFrame(data, columns=['id', 'conversation_id', 'reply_to', 'readability', 'evidence', 'has_reasoning', 'has_reasoning_one_shot', 'politensess', 'relatedness', 'subjectivity', 'polarity', 'text'])


In [16]:
# Assuming your dataframe is named 'df'
count_values_greater_than_08 = df[df['evidence'] > 0.4]['evidence'].count()
print(count_values_greater_than_08)


79418


In [18]:
len(df)

313406

In [17]:
# Assuming your dataframe is named 'df'
count_values_greater_than_08 = df[df['relatedness'] <0.1]['relatedness'].count()
print(count_values_greater_than_08)

156419


In [19]:
import numpy as np

def replace_outliers_with_max_non_outlier(data, threshold=3, percentile_level=99):
    z_scores = (data - np.mean(data)) / np.std(data)
    outliers = np.abs(z_scores) > threshold
    max_non_outlier = np.percentile(data, percentile_level)
    data[outliers] = max_non_outlier
    return data

# Example usage
data = np.array(df['readability'])
cleaned_data = replace_outliers_with_max_non_outlier(data)

print("Original Data:", data)
print("Cleaned Data:", cleaned_data)


Original Data: [-0.5 -0.5 -4.5 ... -1.5  0.  -5.5]
Cleaned Data: [-0.5 -0.5 -4.5 ... -1.5  0.  -5.5]


In [20]:
import numpy as np

def count_values_smaller_than(data, x):
    count = np.count_nonzero(data < x)
    return count

x = -25
count = count_values_smaller_than(cleaned_data, x)

print("Data:", data)
print("Value x:", x)
print("Number of values smaller than x:", count)

Data: [-0.5 -0.5 -4.5 ... -1.5  0.  -5.5]
Value x: -25
Number of values smaller than x: 44


In [21]:
df['new_readability'] = df['readability'].apply(lambda x: -10 if x < -10 else x)

In [22]:
df['has_reasoning'] = df['has_reasoning'].astype(int)

In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['norm_readability'] = scaler.fit_transform(df[['new_readability']])
df['norm_politeness'] = scaler.fit_transform(df[['politensess']])

In [25]:
df.head(20)

,id,conversation_id,reply_to,readability,evidence,has_reasoning,has_reasoning_one_shot,politensess,relatedness,subjectivity,polarity,text,new_readability,norm_readability,norm_politeness
0,xyt38s,xyt38s,None,-0.5,0.5,0,0.785905,0.095238,NaN,0.532381,0.224286,MLB Sat Info\n\nPlay OnAll underdogs with a mo...,-0.5,0.95,0.117647
1,iromk5a,xyt38s,xyt38s,-0.5,0.5,0,0.725411,0.095238,0.887727,0.532381,0.224286,Play OnAll underdogs with a money line of +100...,-0.5,0.95,0.117647
2,irpc624,xyt38s,xyt38s,-4.5,0.5,0,0.882337,0.000000,0.031587,0.000000,0.000000,Rons Picks for Sat\nT.B. -105\nTor OV 7/7.5\nS...,-4.5,0.55,0.000000
3,irp3pvs,xyt38s,xyt38s,-5.5,0.0,0,0.968893,0.047619,0.000000,0.400000,0.800000,Guardians win today,-5.5,0.45,0.058824
4,xyt38b,xyt38b,None,-2.5,0.5,1,0.390972,0.380952,NaN,0.415179,0.015476,What is the deal with people referring to spor...,-2.5,0.75,0.470588
5,iril0dg,xyt38b,xyt38b,0.0,0.5,1,0.998367,0.238095,0.262232,0.625000,0.433333,"Answer: It's a variation of saying ""I'm that ...",0.0,1.00,0.294118
6,iritie3,xyt38b,iril0dg,-4.5,0.0,0,0.542686,0.095238,0.130049,0.700000,-0.400000,As someone who has a daughter who watches a lo...,-4.5,0.55,0.117647
7,irj304y,xyt38b,iril0dg,-4.5,0.0,0,0.678813,0.047619,0.022900,0.000000,0.000000,"What does ""HIM"" stand for?",-4.5,0.55,0.058824
8,irjtg1m,xyt38b,iril0dg,0.0,0.0,0,0.793378,0.095238,0.042969,0.000000,0.000000,"If they press their hand against their liver, ...",0.0,1.00,0.117647
9,irs6cuu,xyt38b,iril0dg,-4.5,0.0,0,0.455724,0.095238,0.000000,0.600000,0.050000,I am now more confused,-4.5,0.55,0.117647


In [28]:
import pickle

with open(f"{base_path}conversations_with_features_dataframe_one_shot.pkl", 'wb') as file:
    pickle.dump(df, file)